## Error Analysis over Finetuned NLI models

- author: sushma akoju, email: sushmaakoju@arizona.edu

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report

In [4]:
gt = 'CompressedGT'
pred = 'pred_labels'
svo = "Part of Premise/Hypothesis Modified"
mod_type_col = 'Modifier Type'
output_path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/june12"
model_folders = ["nli-deberta-v3-base", "roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"]
files = []
files = [os.path.join(os.path.join(output_path, model_folders[0]), file) for file in os.listdir(os.path.join(output_path,model_folders[0])) ]
temp = [os.path.join(os.path.join(output_path, model_folders[1]), file) for file in os.listdir(os.path.join(output_path,model_folders[1])) ]
files.extend(temp)

In [5]:
model_names = ['nli-deberta-v3-base','ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli']

In [6]:
model1_files = []
model2_files = []

for file in files:
  if "8" in file and "32" in file and "deberta" in file:
    model1_files.append(file)
  if "8" in file and "32" in file and "roberta" in file:
    model2_files.append(file)

In [9]:
folds = []
for m1, m2, i in zip(model1_files, model2_files, range(5)):
  print(m1)
  df1 = pd.read_csv(m1, encoding="utf-8")
  df2 = pd.read_csv(m2)
  df1 = df1.rename(columns={'predictions':model_names[0], 'pred_labels':"pred-labels-deberta","Quantifier Type": "Modifier Type"})
  df2 = df2.rename(columns={'predictions':model_names[1], 'pred_labels':"pred-labels-roberta", "Quantifier Type": "Modifier Type"})
  # print(df1.columns, df2.columns)
  df1['fold'] = i

  df1 = df1[['fold','Premise', 'Hypothesis', 'CompressedGT',
       'Modifier Type', 'Modifier', 'Premise/Hypothesis/Both',
       'Part of Premise/Hypothesis Modified', 'nli-deberta-v3-base','text', "pred-labels-deberta"]]
  df2 = df2[['Premise', 'Hypothesis', 'CompressedGT',
       'Modifier Type', 'Modifier', 'Premise/Hypothesis/Both',
       'Part of Premise/Hypothesis Modified', 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
       "pred-labels-roberta"]]

  res = pd.concat([df1, df2[['ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',"pred-labels-roberta"]]], axis='columns')
  folds.append(res)
  # df1[ (df1['Premise']==df2['Premise']) & (df1['Hypothesis']==df2['Hypothesis']) ]['ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli']
  # folds.append(df1.merge(df2, on='text', how='left'))

/content/drive/MyDrive/Colab Notebooks/natural-logic/june12/nli-deberta-v3-base/five_nli-deberta-v3-base_8_32_0_test.xlsx
/content/drive/MyDrive/Colab Notebooks/natural-logic/june12/nli-deberta-v3-base/five_nli-deberta-v3-base_8_32_1_test.xlsx
/content/drive/MyDrive/Colab Notebooks/natural-logic/june12/nli-deberta-v3-base/five_nli-deberta-v3-base_8_32_2_test.xlsx
/content/drive/MyDrive/Colab Notebooks/natural-logic/june12/nli-deberta-v3-base/five_nli-deberta-v3-base_8_32_3_test.xlsx
/content/drive/MyDrive/Colab Notebooks/natural-logic/june12/nli-deberta-v3-base/five_nli-deberta-v3-base_8_32_4_test.xlsx


In [10]:
df1 = df1.rename(columns={"Quantifier Type": "Modifier Type"})
df2 = df2.rename(columns={"Quantifier Type": "Modifier Type"})

In [11]:
df1 = df1.rename(columns={"predictions":model_names[0]})
df2 = df2.rename(columns={"predictions":model_names[1]})

In [12]:
res = pd.concat(folds)

In [13]:
len(res)

1305

In [14]:
res.to_csv(os.path.join(output_path, "combined_finetuned_results.csv"))

In [15]:
res.columns

Index(['fold', 'Premise', 'Hypothesis', 'CompressedGT', 'Modifier Type',
       'Modifier', 'Premise/Hypothesis/Both',
       'Part of Premise/Hypothesis Modified', 'nli-deberta-v3-base', 'text',
       'pred-labels-deberta',
       'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
       'pred-labels-roberta'],
      dtype='object')

In [16]:
res1 = res[res['Modifier Type'] != "No Modifiers"]

In [17]:
res['Modifier Type'].unique()

array(['No Modifiers', 'Universal', 'Existential', 'Negation',
       'Adjectives/Adverbs'], dtype=object)

In [18]:
res1.groupby(["Modifier Type"]).count().reset_index()[["Modifier Type","CompressedGT"]]

,Modifier Type,CompressedGT
0,Adjectives/Adverbs,602
1,Existential,303
2,Negation,167
3,Universal,217


In [19]:
path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/final-datasets/block-wise-data/blocks-dec26/data/15"

data = pd.read_csv(os.path.join(path, "SICCK-zero-shot-analysis-apr24.csv"))
data.groupby(["Modifier Type"]).count().reset_index()[["Modifier Type","CompressedGT"]]

,Modifier Type,CompressedGT
0,Adjectives/Adverbs,602
1,Existential,303
2,Negation,167
3,No Modifiers,15
4,Universal,217


In [20]:
sum(data.groupby(["Modifier Type"]).count().reset_index()["CompressedGT"].tolist())

1304

In [21]:
data["Part of Premise/Hypothesis Modified"]=data["Part of Premise/Hypothesis Modified"].str.lower()
data["Part of Premise/Hypothesis Modified"].unique()

array(['none', 'subject', 'verb', 'object'], dtype=object)

In [22]:
data.groupby(["Part of Premise/Hypothesis Modified"]).count().reset_index()[["Part of Premise/Hypothesis Modified","CompressedGT"]]

,Part of Premise/Hypothesis Modified,CompressedGT
0,none,15
1,object,509
2,subject,560
3,verb,220


In [23]:
r = data.groupby(["Modifier", "CompressedGT"]).count().reset_index()[["Modifier","CompressedGT"]]

In [24]:
r

,Modifier,CompressedGT
0,NONE,Contradiction
1,NONE,FE
2,NONE,Neutral
3,abnormally,Contradiction
4,abnormally,FE
...,...,...
64,sad,Neutral
65,some,Contradiction
66,some,FE
67,some,Neutral


In [25]:
res['Part of Premise/Hypothesis Modified'] = res['Part of Premise/Hypothesis Modified'].str.lower().str.strip()

In [26]:
res['CompressedGT'].value_counts(), res["Modifier Type"].value_counts()

(Neutral          593
 Contradiction    423
 FE               259
 RE                30
 Name: CompressedGT, dtype: int64,
 Adjectives/Adverbs    602
 Existential           303
 Universal             217
 Negation              167
 No Modifiers           16
 Name: Modifier Type, dtype: int64)

## Neutral labels with Negation Modifiers

In [27]:
res['nli-deberta-v3-base'].unique()

array([0, 1, 2])

In [28]:
gp = res.groupby(["Modifier Type","Part of Premise/Hypothesis Modified"]).count().reset_index()
gp

,Modifier Type,Part of Premise/Hypothesis Modified,fold,Premise,Hypothesis,CompressedGT,Modifier,Premise/Hypothesis/Both,nli-deberta-v3-base,text,pred-labels-deberta,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
0,Adjectives/Adverbs,object,301,301,301,301,301,301,301,60,301,301,301
1,Adjectives/Adverbs,subject,211,211,211,211,211,211,211,45,211,211,211
2,Adjectives/Adverbs,verb,90,90,90,90,90,90,90,18,90,90,90
3,Existential,object,126,126,126,126,126,126,126,21,126,126,126
4,Existential,subject,177,177,177,177,177,177,177,36,177,177,177
5,Negation,object,40,40,40,40,40,40,40,6,40,40,40
6,Negation,subject,86,86,86,86,86,86,86,18,86,86,86
7,Negation,verb,41,41,41,41,41,41,41,9,41,41,41
8,No Modifiers,none,16,16,16,16,16,16,16,4,16,16,16
9,Universal,object,42,42,42,42,42,42,42,9,42,42,42


In [29]:
gp = res.groupby(["Modifier Type","Part of Premise/Hypothesis Modified","CompressedGT"]).count().reset_index()
gp[(gp["Modifier Type"] == "Negation") & gp["Modifier Type"]]

,Modifier Type,Part of Premise/Hypothesis Modified,CompressedGT,fold,Premise,Hypothesis,Modifier,Premise/Hypothesis/Both,nli-deberta-v3-base,text,pred-labels-deberta,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
17,Negation,object,Contradiction,10,10,10,10,10,10,0,10,10,10
18,Negation,object,FE,5,5,5,5,5,5,0,5,5,5
19,Negation,object,Neutral,22,22,22,22,22,22,6,22,22,22
20,Negation,object,RE,3,3,3,3,3,3,0,3,3,3
21,Negation,subject,Contradiction,7,7,7,7,7,7,0,7,7,7
22,Negation,subject,FE,8,8,8,8,8,8,0,8,8,8
23,Negation,subject,Neutral,65,65,65,65,65,65,18,65,65,65
24,Negation,subject,RE,6,6,6,6,6,6,0,6,6,6
25,Negation,verb,Contradiction,6,6,6,6,6,6,0,6,6,6
26,Negation,verb,FE,2,2,2,2,2,2,0,2,2,2


In [30]:
gp = res.groupby(["Modifier Type","Part of Premise/Hypothesis Modified", "CompressedGT"]).count().reset_index()
gp[ (gp['Modifier Type']=="Negation") ]

,Modifier Type,Part of Premise/Hypothesis Modified,CompressedGT,fold,Premise,Hypothesis,Modifier,Premise/Hypothesis/Both,nli-deberta-v3-base,text,pred-labels-deberta,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
17,Negation,object,Contradiction,10,10,10,10,10,10,0,10,10,10
18,Negation,object,FE,5,5,5,5,5,5,0,5,5,5
19,Negation,object,Neutral,22,22,22,22,22,22,6,22,22,22
20,Negation,object,RE,3,3,3,3,3,3,0,3,3,3
21,Negation,subject,Contradiction,7,7,7,7,7,7,0,7,7,7
22,Negation,subject,FE,8,8,8,8,8,8,0,8,8,8
23,Negation,subject,Neutral,65,65,65,65,65,65,18,65,65,65
24,Negation,subject,RE,6,6,6,6,6,6,0,6,6,6
25,Negation,verb,Contradiction,6,6,6,6,6,6,0,6,6,6
26,Negation,verb,FE,2,2,2,2,2,2,0,2,2,2


In [31]:
pd.set_option('display.max_rows', None)

In [32]:
gp[ (gp['Modifier Type']=="Adjectives/Adverbs") | (gp['Modifier Type']=="Negation") ]

,Modifier Type,Part of Premise/Hypothesis Modified,CompressedGT,fold,Premise,Hypothesis,Modifier,Premise/Hypothesis/Both,nli-deberta-v3-base,text,pred-labels-deberta,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
0,Adjectives/Adverbs,object,Contradiction,112,112,112,112,112,112,21,112,112,112
1,Adjectives/Adverbs,object,FE,64,64,64,64,64,64,0,64,64,64
2,Adjectives/Adverbs,object,Neutral,125,125,125,125,125,125,39,125,125,125
3,Adjectives/Adverbs,subject,Contradiction,77,77,77,77,77,77,15,77,77,77
4,Adjectives/Adverbs,subject,FE,41,41,41,41,41,41,0,41,41,41
5,Adjectives/Adverbs,subject,Neutral,93,93,93,93,93,93,30,93,93,93
6,Adjectives/Adverbs,verb,Contradiction,36,36,36,36,36,36,6,36,36,36
7,Adjectives/Adverbs,verb,FE,19,19,19,19,19,19,0,19,19,19
8,Adjectives/Adverbs,verb,Neutral,35,35,35,35,35,35,12,35,35,35
17,Negation,object,Contradiction,10,10,10,10,10,10,0,10,10,10


In [33]:
gp[ (gp['Modifier Type']=="Existential") ]

,Modifier Type,Part of Premise/Hypothesis Modified,CompressedGT,fold,Premise,Hypothesis,Modifier,Premise/Hypothesis/Both,nli-deberta-v3-base,text,pred-labels-deberta,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
9,Existential,object,Contradiction,32,32,32,32,32,32,7,32,32,32
10,Existential,object,FE,37,37,37,37,37,37,0,37,37,37
11,Existential,object,Neutral,55,55,55,55,55,55,14,55,55,55
12,Existential,object,RE,2,2,2,2,2,2,0,2,2,2
13,Existential,subject,Contradiction,56,56,56,56,56,56,8,56,56,56
14,Existential,subject,FE,43,43,43,43,43,43,0,43,43,43
15,Existential,subject,Neutral,75,75,75,75,75,75,28,75,75,75
16,Existential,subject,RE,3,3,3,3,3,3,0,3,3,3


## Universal and Existential Quantifiers & the Contradiction

In [34]:
gp = res.groupby(["Modifier Type","Part of Premise/Hypothesis Modified", "CompressedGT", "pred-labels-deberta", "Premise/Hypothesis/Both"]).count().reset_index()
gp[ (gp['Modifier Type']=="Universal") ]

,Modifier Type,Part of Premise/Hypothesis Modified,CompressedGT,pred-labels-deberta,Premise/Hypothesis/Both,fold,Premise,Hypothesis,Modifier,nli-deberta-v3-base,text,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
168,Universal,object,Contradiction,Negation,Both,5,5,5,5,5,1,5,5
169,Universal,object,Contradiction,Negation,Hypothesis,4,4,4,4,4,1,4,4
170,Universal,object,Contradiction,Negation,Premise,5,5,5,5,5,1,5,5
171,Universal,object,FE,Neutral,Both,3,3,3,3,3,0,3,3
172,Universal,object,FE,Neutral,Premise,3,3,3,3,3,0,3,3
173,Universal,object,FE,RE,Premise,2,2,2,2,2,0,2,2
174,Universal,object,Neutral,FE,Both,3,3,3,3,3,1,3,3
175,Universal,object,Neutral,FE,Hypothesis,5,5,5,5,5,2,5,5
176,Universal,object,Neutral,FE,Premise,2,2,2,2,2,1,2,2
177,Universal,object,Neutral,Neutral,Both,1,1,1,1,1,1,1,1


In [35]:
gp[ (gp['Modifier Type']=="Universal") & (gp['CompressedGT'] == "Contradiction") &(gp['pred-labels-deberta'] == "Contradiction")  ]['fold'].sum()

0

In [36]:
gp[ (gp['Modifier Type']=="Universal") & (gp['CompressedGT'] == "Contradiction")  ]['fold'].sum()

80

In [37]:
gp[ (gp['Modifier Type']=="Existential") ]

,Modifier Type,Part of Premise/Hypothesis Modified,CompressedGT,pred-labels-deberta,Premise/Hypothesis/Both,fold,Premise,Hypothesis,Modifier,nli-deberta-v3-base,text,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
55,Existential,object,Contradiction,FE,Both,1,1,1,1,1,1,1,1
56,Existential,object,Contradiction,FE,Hypothesis,2,2,2,2,2,0,2,2
57,Existential,object,Contradiction,Negation,Both,11,11,11,11,11,2,11,11
58,Existential,object,Contradiction,Negation,Hypothesis,8,8,8,8,8,2,8,8
59,Existential,object,Contradiction,Negation,Premise,10,10,10,10,10,2,10,10
60,Existential,object,FE,FE,Both,3,3,3,3,3,0,3,3
61,Existential,object,FE,FE,Hypothesis,4,4,4,4,4,0,4,4
62,Existential,object,FE,FE,Premise,9,9,9,9,9,0,9,9
63,Existential,object,FE,Neutral,Both,10,10,10,10,10,0,10,10
64,Existential,object,FE,Neutral,Hypothesis,2,2,2,2,2,0,2,2


In [38]:
gp[ (gp['Modifier Type']=="Existential") & (gp['CompressedGT'] == "Contradiction") &(gp['pred-labels-deberta'] == "Contradiction")  ]['fold'].sum()

0

In [39]:
gp[ (gp['Modifier Type']=="Existential") & (gp['CompressedGT'] == "Contradiction")  ]['fold'].sum()

88

In [40]:
gp[ (gp['Modifier Type']=="No Modifiers") ]

,Modifier Type,Part of Premise/Hypothesis Modified,CompressedGT,pred-labels-deberta,Premise/Hypothesis/Both,fold,Premise,Hypothesis,Modifier,nli-deberta-v3-base,text,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
163,No Modifiers,none,Contradiction,Negation,NONE,7,7,7,7,7,2,7,7
164,No Modifiers,none,FE,FE,NONE,1,1,1,1,1,0,1,1
165,No Modifiers,none,FE,Neutral,NONE,4,4,4,4,4,0,4,4
166,No Modifiers,none,Neutral,FE,NONE,3,3,3,3,3,1,3,3
167,No Modifiers,none,Neutral,Neutral,NONE,1,1,1,1,1,1,1,1


In [41]:
gp[ (gp['Modifier Type']=="Negation") ]

,Modifier Type,Part of Premise/Hypothesis Modified,CompressedGT,pred-labels-deberta,Premise/Hypothesis/Both,fold,Premise,Hypothesis,Modifier,nli-deberta-v3-base,text,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
108,Negation,object,Contradiction,FE,Both,1,1,1,1,1,0,1,1
109,Negation,object,Contradiction,FE,Hypothesis,1,1,1,1,1,0,1,1
110,Negation,object,Contradiction,Negation,Both,1,1,1,1,1,0,1,1
111,Negation,object,Contradiction,Negation,Hypothesis,4,4,4,4,4,0,4,4
112,Negation,object,Contradiction,Negation,Premise,2,2,2,2,2,0,2,2
113,Negation,object,Contradiction,RE,Both,1,1,1,1,1,0,1,1
114,Negation,object,FE,FE,Both,1,1,1,1,1,0,1,1
115,Negation,object,FE,FE,Premise,3,3,3,3,3,0,3,3
116,Negation,object,FE,Neutral,Both,1,1,1,1,1,0,1,1
117,Negation,object,Neutral,FE,Both,5,5,5,5,5,2,5,5


In [42]:
gp[ (gp['Modifier Type']=="Negation") & (gp['CompressedGT'] == "Neutral") & (gp['pred-labels-deberta'] != "Neutral")  ]['fold'].sum()

115

In [43]:
gp[ (gp['Modifier Type']=="Negation")]["fold"].sum()

167

In [44]:
1- (105/167)

0.37125748502994016

## Verb Modified Examples

In [ ]:
# gp = res.groupby(["Modifier Type","Part of Premise/Hypothesis Modified", "pred-labels-roberta", "CompressedGT"]).count().reset_index()
# gp[ (gp['Modifier Type']=="Negation") & (gp["CompressedGT"] == "Neutral")]

In [45]:
gp[(gp['Part of Premise/Hypothesis Modified'] == "verb") & (gp['Modifier Type']=="Negation") & (gp['pred-labels-deberta'] != "Neutral") & (gp["CompressedGT"] == "Neutral")]['fold'].sum()

31

In [46]:
gp[(gp['Part of Premise/Hypothesis Modified'] == "verb") & (gp['Modifier Type']=="Negation") & (gp["CompressedGT"] == "Neutral")]['fold'].sum()

31

In [47]:
gp[(gp['Part of Premise/Hypothesis Modified'] == "verb") & (gp['Modifier Type']=="Universal") & (gp['pred-labels-deberta'] != "Neutral") & (gp["CompressedGT"] == "Neutral")]['fold'].sum()

45

In [48]:
gp[(gp['Part of Premise/Hypothesis Modified'] == "verb") & (gp['Modifier Type']=="Universal") & (gp["CompressedGT"] == "Neutral")]['fold'].sum()

49

In [49]:
34/49

0.6938775510204082

In [50]:
gp[ (gp["Part of Premise/Hypothesis Modified"]=="verb") ]

,Modifier Type,Part of Premise/Hypothesis Modified,CompressedGT,pred-labels-deberta,Premise/Hypothesis/Both,fold,Premise,Hypothesis,Modifier,nli-deberta-v3-base,text,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
39,Adjectives/Adverbs,verb,Contradiction,FE,Hypothesis,2,2,2,2,2,0,2,2
40,Adjectives/Adverbs,verb,Contradiction,Negation,Both,12,12,12,12,12,2,12,12
41,Adjectives/Adverbs,verb,Contradiction,Negation,Hypothesis,10,10,10,10,10,2,10,10
42,Adjectives/Adverbs,verb,Contradiction,Negation,Premise,12,12,12,12,12,2,12,12
43,Adjectives/Adverbs,verb,FE,FE,Premise,3,3,3,3,3,0,3,3
44,Adjectives/Adverbs,verb,FE,Neutral,Both,10,10,10,10,10,0,10,10
45,Adjectives/Adverbs,verb,FE,Neutral,Premise,2,2,2,2,2,0,2,2
46,Adjectives/Adverbs,verb,FE,RE,Premise,4,4,4,4,4,0,4,4
47,Adjectives/Adverbs,verb,Neutral,FE,Both,5,5,5,5,5,2,5,5
48,Adjectives/Adverbs,verb,Neutral,FE,Hypothesis,16,16,16,16,16,4,16,16


In [51]:
data.groupby(["Part of Premise/Hypothesis Modified"]).count().reset_index()[["Part of Premise/Hypothesis Modified","CompressedGT"]]

,Part of Premise/Hypothesis Modified,CompressedGT
0,none,15
1,object,509
2,subject,560
3,verb,220


In [52]:
res['pred-labels-roberta'][res['pred-labels-roberta'] == 'Negation'] = 'Contradiction'
res['pred-labels-deberta'][res['pred-labels-deberta'] == 'Negation'] = 'Contradiction'

res['pred-labels-roberta'].unique(), res['pred-labels-deberta'].unique()

<ipython-input-52-03f3fdce7289>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['pred-labels-roberta'][res['pred-labels-roberta'] == 'Negation'] = 'Contradiction'
<ipython-input-52-03f3fdce7289>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['pred-labels-deberta'][res['pred-labels-deberta'] == 'Negation'] = 'Contradiction'


(array(['Neutral', 'RE', 'Contradiction', 'FE'], dtype=object),
 array(['Contradiction', 'FE', 'Neutral', 'RE'], dtype=object))

In [53]:
len(res[res['CompressedGT']=='Contradiction'])

423

In [54]:
gp = res.groupby(['CompressedGT','Modifier Type','pred-labels-roberta']).count().reset_index()
gp[gp['CompressedGT']=='Contradiction']

,CompressedGT,Modifier Type,pred-labels-roberta,fold,Premise,Hypothesis,Modifier,Premise/Hypothesis/Both,Part of Premise/Hypothesis Modified,nli-deberta-v3-base,text,pred-labels-deberta,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli
0,Contradiction,Adjectives/Adverbs,Contradiction,149,149,149,149,149,149,149,1,149,149
1,Contradiction,Adjectives/Adverbs,FE,5,5,5,5,5,5,5,5,5,5
2,Contradiction,Adjectives/Adverbs,Neutral,68,68,68,68,68,68,68,33,68,68
3,Contradiction,Adjectives/Adverbs,RE,3,3,3,3,3,3,3,3,3,3
4,Contradiction,Existential,Contradiction,68,68,68,68,68,68,68,0,68,68
5,Contradiction,Existential,Neutral,19,19,19,19,19,19,19,14,19,19
6,Contradiction,Existential,RE,1,1,1,1,1,1,1,1,1,1
7,Contradiction,Negation,Contradiction,11,11,11,11,11,11,11,0,11,11
8,Contradiction,Negation,FE,3,3,3,3,3,3,3,0,3,3
9,Contradiction,Negation,Neutral,9,9,9,9,9,9,9,0,9,9


In [55]:
res.groupby(['CompressedGT', 'Part of Premise/Hypothesis Modified']).count().reset_index()

,CompressedGT,Part of Premise/Hypothesis Modified,fold,Premise,Hypothesis,Modifier Type,Modifier,Premise/Hypothesis/Both,nli-deberta-v3-base,text,pred-labels-deberta,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
0,Contradiction,none,7,7,7,7,7,7,7,2,7,7,7
1,Contradiction,object,168,168,168,168,168,168,168,31,168,168,168
2,Contradiction,subject,182,182,182,182,182,182,182,35,182,182,182
3,Contradiction,verb,66,66,66,66,66,66,66,9,66,66,66
4,FE,none,5,5,5,5,5,5,5,0,5,5,5
5,FE,object,114,114,114,114,114,114,114,0,114,114,114
6,FE,subject,106,106,106,106,106,106,106,0,106,106,106
7,FE,verb,34,34,34,34,34,34,34,0,34,34,34
8,Neutral,none,4,4,4,4,4,4,4,2,4,4,4
9,Neutral,object,217,217,217,217,217,217,217,65,217,217,217


In [56]:
gp = res.groupby(["Modifier Type","Modifier","Part of Premise/Hypothesis Modified", "CompressedGT", "pred-labels-deberta", "Premise/Hypothesis/Both"]).count().reset_index()
gp[ (gp['Modifier Type']=="Universal") ]

,Modifier Type,Modifier,Part of Premise/Hypothesis Modified,CompressedGT,pred-labels-deberta,Premise/Hypothesis/Both,fold,Premise,Hypothesis,nli-deberta-v3-base,text,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
399,Universal,always,verb,Contradiction,Contradiction,Both,6,6,6,6,1,6,6
400,Universal,always,verb,Contradiction,Contradiction,Hypothesis,5,5,5,5,1,5,5
401,Universal,always,verb,Contradiction,Contradiction,Premise,6,6,6,6,1,6,6
402,Universal,always,verb,Contradiction,FE,Hypothesis,1,1,1,1,0,1,1
403,Universal,always,verb,FE,FE,Both,1,1,1,1,0,1,1
404,Universal,always,verb,FE,FE,Hypothesis,1,1,1,1,0,1,1
405,Universal,always,verb,FE,FE,Premise,1,1,1,1,0,1,1
406,Universal,always,verb,FE,Neutral,Both,4,4,4,4,0,4,4
407,Universal,always,verb,FE,Neutral,Premise,1,1,1,1,0,1,1
408,Universal,always,verb,FE,RE,Premise,2,2,2,2,0,2,2


In [57]:
gp[ (gp['Modifier Type']=="Existential") ]

,Modifier Type,Modifier,Part of Premise/Hypothesis Modified,CompressedGT,pred-labels-deberta,Premise/Hypothesis/Both,fold,Premise,Hypothesis,nli-deberta-v3-base,text,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
219,Existential,all but one,object,Contradiction,Contradiction,Both,1,1,1,1,0,1,1
220,Existential,all but one,object,Contradiction,FE,Both,1,1,1,1,1,1,1
221,Existential,all but one,object,FE,FE,Both,2,2,2,2,0,2,2
222,Existential,all but one,object,FE,FE,Premise,5,5,5,5,0,5,5
223,Existential,all but one,object,FE,Neutral,Both,1,1,1,1,0,1,1
224,Existential,all but one,object,FE,Neutral,Premise,1,1,1,1,0,1,1
225,Existential,all but one,object,FE,RE,Premise,1,1,1,1,0,1,1
226,Existential,all but one,object,Neutral,Contradiction,Hypothesis,1,1,1,1,0,1,1
227,Existential,all but one,object,Neutral,Contradiction,Premise,1,1,1,1,0,1,1
228,Existential,all but one,object,Neutral,FE,Both,9,9,9,9,2,9,9


In [58]:
gp[ (gp['Modifier Type']=="Negation") ]

,Modifier Type,Modifier,Part of Premise/Hypothesis Modified,CompressedGT,pred-labels-deberta,Premise/Hypothesis/Both,fold,Premise,Hypothesis,nli-deberta-v3-base,text,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pred-labels-roberta
334,Negation,no,object,Contradiction,Contradiction,Both,1,1,1,1,0,1,1
335,Negation,no,object,Contradiction,Contradiction,Hypothesis,4,4,4,4,0,4,4
336,Negation,no,object,Contradiction,Contradiction,Premise,2,2,2,2,0,2,2
337,Negation,no,object,Contradiction,FE,Both,1,1,1,1,0,1,1
338,Negation,no,object,Contradiction,FE,Hypothesis,1,1,1,1,0,1,1
339,Negation,no,object,Contradiction,RE,Both,1,1,1,1,0,1,1
340,Negation,no,object,FE,FE,Both,1,1,1,1,0,1,1
341,Negation,no,object,FE,FE,Premise,3,3,3,3,0,3,3
342,Negation,no,object,FE,Neutral,Both,1,1,1,1,0,1,1
343,Negation,no,object,Neutral,Contradiction,Both,2,2,2,2,0,2,2
